<a href="https://colab.research.google.com/github/szheng3/recommendation-system/blob/main/Explore_CQL/DLR2/HM_SA2C_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SA2C-SASRec Recommender on H&M Data with Conservative Q-learning

In this notebook we train an SA2C_SASrec model without and with CQL to train for recommendation on H&M dataset.

1. Clone the git repository containing all the source code

In [2]:
# cloning the git repository
!git clone https://github.com/szheng3/recommendation-system

Cloning into 'recommendation-system'...
remote: Enumerating objects: 1163, done.
remote: Counting objects: 100% (430/430), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 1163 (delta 219), reused 325 (delta 162), pack-reused 733
Receiving objects: 100% (1163/1163), 10.55 MiB | 15.96 MiB/s, done.
Resolving deltas: 100% (565/565), done.


2. Install the missing libraries (specifically for google colab)

In [3]:
# installing missing libraries (specifically for google colab)
!pip install trfl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 916.6 kB/s eta 0:00:00


3. Download the H&M Dataset transaction log file from AWS S3 bucket

In [4]:
# downloading the "transactions_train.csv" datafile from the H&M Dataset
!wget https://aipi590.s3.amazonaws.com/transactions_train.csv -P "/content/recommendation-system/Explore_CQL/Data/HM_data/"

--2023-05-02 20:26:57--  https://aipi590.s3.amazonaws.com/transactions_train.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 52.217.93.100, 54.231.199.217, 52.217.47.180, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|52.217.93.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488002253 (3.2G) [text/csv]
Saving to: ‘/content/recommendation-system/Explore_CQL/Data/HM_data/transactions_train.csv’

transactions_train. 100%[===================>]   3.25G  27.6MB/s    in 76s     

2023-05-02 20:28:13 (43.6 MB/s) - ‘/content/recommendation-system/Explore_CQL/Data/HM_data/transactions_train.csv’ saved [3488002253/3488002253]



4. Run the "gen_replay_buffer.py" script to pre-process data and generate replay buffer

In [5]:
# Executing the python script "gen_replay_buffer" to sample the given H&M dataset
!python "/content/recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer_HM.py" --data="/content/recommendation-system/Explore_CQL/Data/HM_data"


Start reading all transaction data ...
Finish reading in 00:00:28

Filter and save all valid sampled data
Index(['timestamp', 'session_id', 'item_id', 'price', 'sales_channel_id'], dtype='object')

Start counting popularity ...
13040912it [08:16, 26272.28it/s]
Popularity finished in 00:08:16

Start spliting into train, val, test data ...

           Generate Replay Buffer:
                Total Session Size : 1245612
                     Train:      871928 ids | 9124752 actions
                     Validation: 249122 ids | 2611174 actions
                     Test:       124562 ids | 1304986 actions
                     
                Random : True
                Random Seed : 1234
                Format : paper
    
                Total session id number : 1245612
                Total item id number  : 96222
    
Generating training replay buffer
100% 608701/608701 [09:45<00:00, 1038.91it/s]


5. Run script to begin training and evaluate the SASRec-SNQN model.

In [6]:
# Executing the python script "SA2C_v3.py" to train a SA2C-SASrec model on the sampled data and evaluate it on validation data using HR (hit rate) and NDCG metrics for different values of k
!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v3_5.py" --model=SASRec --data="/content/recommendation-system/Explore_CQL/Data/HM_data/" --epoch=5

2023-05-02 20:48:39.698824: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-02 20:48:39.752100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 20:48:40.822309: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Not using CQL loss
2023-05-02 20:48:42.965890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed 

In [7]:
# Executing the python script "SA2C_v3.py" to train a SA2C-SASrec model on the sampled data with CQL and evaluate it on validation data using HR (hit rate) and NDCG metrics for different values of k
!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v3_5.py" --model=SASRec --CQL_alpha=1.0 --data="/content/recommendation-system/Explore_CQL/Data/HM_data/" --epoch=5

2023-05-03 00:17:09.045701: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 00:17:09.098977: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 00:17:10.140498: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: SA2C_v3.py
       [-h]
       [--epoch EPOCH]
       [--data [DATA]]
       [--batch_size BATCH_SIZE]
       [--hidden_factor HIDDEN_FACTOR]
       [--r_click R_CLICK]
       [--r_buy R_BUY]
       [--r_negative R_NEGATIVE]
       [--lr LR]
  